In [ ]:
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime
import time

# .env 파일 로드
load_dotenv()
api_key = os.getenv("YOUTUBE_API_KEY")

if not api_key:
    raise ValueError("YouTube API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

youtube = build("youtube", "v3", developerKey=api_key)


def search_videos(keyword, max_results=10, order="relevance"):
    """
    키워드로 유튜브 영상 검색
    
    Args:
        keyword (str): 검색 키워드
        max_results (int): 검색할 영상 개수 (최대 50)
        order (str): 정렬 방식 - 'relevance'(관련성), 'date'(최신순), 'viewCount'(조회수)
    
    Returns:
        list: 영상 정보 리스트
    """
    print(f"🔍 '{keyword}' 키워드로 영상 검색 중...")
    
    try:
        search_response = youtube.search().list(
            q=keyword,
            part="snippet",
            maxResults=min(max_results, 50),
            type="video",
            order=order,
            regionCode="KR"  # 한국 지역 설정
        ).execute()
        
        video_ids = [item["id"]["videoId"] for item in search_response["items"]]
        print(f"✅ {len(video_ids)}개의 영상을 찾았습니다.")
        
        return video_ids
    
    except Exception as e:
        print(f"❌ 검색 오류: {e}")
        return []


def get_video_details(video_ids):
    """
    영상의 상세 정보 가져오기 (조회수, 좋아요, 댓글수 등)
    
    Args:
        video_ids (list): 영상 ID 리스트
    
    Returns:
        pd.DataFrame: 영상 정보 데이터프레임
    """
    print(f"📊 {len(video_ids)}개 영상의 상세 정보 수집 중...")
    
    videos_data = []
    
    # YouTube API는 한 번에 최대 50개까지만 조회 가능
    for i in range(0, len(video_ids), 50):
        batch_ids = video_ids[i:i+50]
        
        try:
            video_response = youtube.videos().list(
                part="snippet,statistics,contentDetails",
                id=",".join(batch_ids)
            ).execute()
            
            for item in video_response["items"]:
                video_info = {
                    "video_id": item["id"],
                    "title": item["snippet"]["title"],
                    "channel": item["snippet"]["channelTitle"],
                    "published_at": item["snippet"]["publishedAt"],
                    "description": item["snippet"]["description"],
                    "view_count": int(item["statistics"].get("viewCount", 0)),
                    "like_count": int(item["statistics"].get("likeCount", 0)),
                    "comment_count": int(item["statistics"].get("commentCount", 0)),
                    "duration": item["contentDetails"]["duration"],
                    "tags": ", ".join(item["snippet"].get("tags", [])),
                    "url": f"https://www.youtube.com/watch?v={item['id']}"
                }
                videos_data.append(video_info)
        
        except Exception as e:
            print(f"❌ 영상 정보 수집 오류: {e}")
            continue
    
    df = pd.DataFrame(videos_data)
    print(f"✅ {len(df)}개 영상 정보 수집 완료")
    
    return df


def get_video_comments(video_id, max_comments=100):
    """
    특정 영상의 댓글 가져오기
    
    Args:
        video_id (str): 영상 ID
        max_comments (int): 가져올 최대 댓글 수
    
    Returns:
        list: 댓글 정보 리스트
    """
    comments = []
    next_page_token = None
    
    try:
        while len(comments) < max_comments:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id,
                maxResults=min(100, max_comments - len(comments)),
                pageToken=next_page_token,
                textFormat="plainText",
                order="relevance"  # 'time'(최신순) 또는 'relevance'(관련성순)
            )
            response = request.execute()
            
            for item in response["items"]:
                top_comment = item["snippet"]["topLevelComment"]["snippet"]
                
                comment_info = {
                    "comment_id": item["snippet"]["topLevelComment"]["id"],
                    "video_id": video_id,
                    "author": top_comment["authorDisplayName"],
                    "text": top_comment["textDisplay"],
                    "like_count": top_comment["likeCount"],
                    "published_at": top_comment["publishedAt"],
                    "reply_count": item["snippet"]["totalReplyCount"]
                }
                comments.append(comment_info)
                
                # 대댓글도 수집 (있는 경우)
                if "replies" in item:
                    for reply in item["replies"]["comments"]:
                        reply_snippet = reply["snippet"]
                        reply_info = {
                            "comment_id": reply["id"],
                            "video_id": video_id,
                            "author": reply_snippet["authorDisplayName"],
                            "text": reply_snippet["textDisplay"],
                            "like_count": reply_snippet["likeCount"],
                            "published_at": reply_snippet["publishedAt"],
                            "reply_count": 0,
                            "is_reply": True,
                            "parent_id": comment_info["comment_id"]
                        }
                        comments.append(reply_info)
            
            next_page_token = response.get("nextPageToken")
            
            if not next_page_token:
                break
            
            time.sleep(0.5)  # API 호출 제한 방지
    
    except Exception as e:
        if "commentsDisabled" in str(e):
            print(f"  ⚠️  댓글이 비활성화된 영상입니다.")
        else:
            print(f"  ❌ 댓글 수집 오류: {e}")
    
    return comments


def collect_all_comments(video_ids, videos_df, max_comments_per_video=100):
    """
    여러 영상의 댓글을 일괄 수집 (영상 제목 포함)
    
    Args:
        video_ids (list): 영상 ID 리스트
        videos_df (pd.DataFrame): 영상 정보 데이터프레임
        max_comments_per_video (int): 영상당 최대 댓글 수
    
    Returns:
        pd.DataFrame: 댓글 데이터프레임 (영상 제목 포함)
    """
    print(f"\n💬 {len(video_ids)}개 영상의 댓글 수집 중...")
    
    all_comments = []
    
    # 영상 ID를 키로, 제목과 채널명을 값으로 하는 딕셔너리 생성
    video_info_dict = {}
    for _, row in videos_df.iterrows():
        video_info_dict[row['video_id']] = {
            'title': row['title'],
            'channel': row['channel'],
            'url': row['url']
        }
    
    for idx, video_id in enumerate(video_ids, 1):
        print(f"  [{idx}/{len(video_ids)}] 영상 {video_id} 댓글 수집 중...")
        
        comments = get_video_comments(video_id, max_comments_per_video)
        
        # 각 댓글에 영상 제목과 채널명 추가
        for comment in comments:
            if video_id in video_info_dict:
                comment['video_title'] = video_info_dict[video_id]['title']
                comment['video_channel'] = video_info_dict[video_id]['channel']
                comment['video_url'] = video_info_dict[video_id]['url']
        
        all_comments.extend(comments)
        
        print(f"    ✅ {len(comments)}개 댓글 수집")
        time.sleep(1)  # API 호출 제한 방지
    
    df = pd.DataFrame(all_comments)
    
    # 컬럼 순서 재정렬 (영상 정보를 앞쪽에 배치)
    if not df.empty:
        cols = ['video_id', 'video_title', 'video_channel', 'video_url', 
                'comment_id', 'author', 'text', 'like_count', 
                'published_at', 'reply_count']
        
        # 대댓글 관련 컬럼이 있으면 추가
        if 'is_reply' in df.columns:
            cols.extend(['is_reply', 'parent_id'])
        
        df = df[cols]
    
    print(f"\n✅ 총 {len(df)}개의 댓글 수집 완료")
    
    return df


def save_to_csv(df, filename):
    """데이터프레임을 CSV 파일로 저장"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = f"{filename}_{timestamp}.csv"
    df.to_csv(filepath, index=False, encoding="utf-8-sig")
    print(f"💾 데이터 저장 완료: {filepath}")
    return filepath


# ========================================
# 메인 실행 코드
# ========================================

if __name__ == "__main__":
    # 설정
    KEYWORD = "K-beauty"  # 검색 키워드
    MAX_VIDEOS = 10  # 검색할 영상 개수
    MAX_COMMENTS_PER_VIDEO = 50  # 영상당 댓글 수
    ORDER = "relevance"  # 'relevance', 'date', 'viewCount'
    
    print("=" * 60)
    print("🎥 YouTube 영상 및 댓글 수집 시작")
    print("=" * 60)
    
    # 1. 영상 검색
    video_ids = search_videos(KEYWORD, max_results=MAX_VIDEOS, order=ORDER)
    
    if not video_ids:
        print("❌ 검색된 영상이 없습니다.")
        exit()
    
    # 2. 영상 상세 정보 수집
    videos_df = get_video_details(video_ids)
    
    # 3. 댓글 수집 (영상 정보 전달)
    comments_df = collect_all_comments(video_ids, videos_df, max_comments_per_video=MAX_COMMENTS_PER_VIDEO)
    
    # 4. 데이터 저장
    print("\n" + "=" * 60)
    print("💾 데이터 저장 중...")
    print("=" * 60)
    
    video_file = save_to_csv(videos_df, f"youtube_videos_{KEYWORD}")
    comment_file = save_to_csv(comments_df, f"youtube_comments_{KEYWORD}")
    
    # 5. 기본 통계 출력
    print("\n" + "=" * 60)
    print("📊 수집 결과 요약")
    print("=" * 60)
    print(f"✅ 수집된 영상: {len(videos_df)}개")
    print(f"✅ 수집된 댓글: {len(comments_df)}개")
    print(f"✅ 평균 조회수: {videos_df['view_count'].mean():,.0f}")
    print(f"✅ 평균 좋아요: {videos_df['like_count'].mean():,.0f}")
    print(f"✅ 총 조회수: {videos_df['view_count'].sum():,.0f}")
    
    print("\n" + "=" * 60)
    print("🎉 데이터 수집 완료!")
    print("=" * 60)
    
    # 6. 데이터 미리보기
    print("\n📋 영상 데이터 미리보기:")
    print(videos_df[['title', 'channel', 'view_count', 'comment_count']].head())
    
    print("\n💬 댓글 데이터 미리보기:")
    if not comments_df.empty:
        print(comments_df[['video_title', 'author', 'text', 'like_count']].head())
    else:
        print("수집된 댓글이 없습니다.")

🎥 YouTube 영상 및 댓글 수집 시작
🔍 'K-beauty' 키워드로 영상 검색 중...
✅ 10개의 영상을 찾았습니다.
📊 10개 영상의 상세 정보 수집 중...
✅ 10개 영상 정보 수집 완료

💬 10개 영상의 댓글 수집 중...
  [1/10] 영상 uSCUNfwMOJs 댓글 수집 중...
    ✅ 126개 댓글 수집
  [2/10] 영상 gGeRJWDtgY0 댓글 수집 중...
    ✅ 28개 댓글 수집
  [3/10] 영상 iO8oW6CRiRk 댓글 수집 중...
    ✅ 82개 댓글 수집
  [4/10] 영상 i1o1XILTSxU 댓글 수집 중...
    ✅ 73개 댓글 수집
  [5/10] 영상 48QReWnEPuc 댓글 수집 중...
    ✅ 122개 댓글 수집
  [6/10] 영상 wsB0opxgIFI 댓글 수집 중...
    ✅ 87개 댓글 수집
  [7/10] 영상 vUblA8O08oQ 댓글 수집 중...
    ✅ 55개 댓글 수집
  [8/10] 영상 Bi_8gjx0QB0 댓글 수집 중...
    ✅ 95개 댓글 수집
  [9/10] 영상 3J1j3JmSAgs 댓글 수집 중...
    ✅ 49개 댓글 수집
  [10/10] 영상 UyBMKPlEw0U 댓글 수집 중...
    ✅ 72개 댓글 수집

✅ 총 789개의 댓글 수집 완료

💾 데이터 저장 중...
💾 데이터 저장 완료: youtube_videos_K-beauty_20251103_113247.csv
💾 데이터 저장 완료: youtube_comments_K-beauty_20251103_113247.csv

📊 수집 결과 요약
✅ 수집된 영상: 10개
✅ 수집된 댓글: 789개
✅ 평균 조회수: 3,565,390
✅ 평균 좋아요: 167,167
✅ 총 조회수: 35,653,897

🎉 데이터 수집 완료!

📋 영상 데이터 미리보기:
                                               title     channel  view_c

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 토크나이저와 모델 로드 (첫 실행 시 자동 다운로드)
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 출력 생성
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Wie ich er bitten?


In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 모델을 GPU에 로드
model = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-small",
    device_map="auto"  # GPU가 있으면 자동으로 배치
)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Wie ich er bitten?


In [4]:
import torch
print(torch.__version__)  # 예: 2.7.0
print(torch.cuda.is_available())  # True가 나와야 함
print(torch.version.cuda)  # 설치된 CUDA 버전 확인

2.7.1+cu118
True
11.8
